In [ ]:
import os
os.chdir('..')

In [ ]:
import torch
import numpy as np
from gnn_library.util import load
from evaluate import evaluate_model
from instance_generator import sample_instances
from params import *
from util import _box_plots

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

GNN, args = load('GNN2_hyperparam_tuned', device)

## Box plot baseline evaluation

#### Evaluate CRs on all graph configurations

In [ ]:
rng = np.random.default_rng()
num_trials = 100
batch_size = 500
node_configs = [(30, 10)]
graph_configs = MAIN_TESTING_GRAPH_CONFIGS
baselines_kwargs = {
    'greedy': {},
    'greedy_t': {'threshold': 0.35},
    'lp_rounding': {'rng': rng},
    'naor_lp_rounding': {'rng': rng},
    'pollner_lp_rounding': {'rng': rng}
}

data = {node_config: {} for node_config in node_configs}

for node_config in node_configs:
    for graph_config in graph_configs:
            print(node_config, graph_config)
            instances = sample_instances(
                *node_config,
                num_trials,
                rng,
                args.__dict__,
                **graph_config
            )

            cr_ratios, _ = evaluate_model(
                meta_model=None,
                meta_model_type=None,
                base_models=[],
                instances=instances,
                batch_size=batch_size,
                rng=rng,
                num_realizations=20,
                baselines=['greedy', 'greedy_t', 'lp_rounding', 'naor_lp_rounding', 'pollner_lp_rounding'],
                **baselines_kwargs
            )

            data[node_config][str(graph_config)] = cr_ratios


#### Generate box plots

In [ ]:
results = {}
for graph_type, val in data[node_configs[0]].items():
	results[graph_type] = val

_box_plots(results, lambda graph_type: f"GNN2_classify_{graph_type} {node_config[1]}x{node_config[0]}")

In [ ]:
for label in ['greedy', 'greedy_t', 'lp_rounding', 'naor_lp_rounding', 'pollner_lp_rounding']:
    print(label)
    print(np.nanmean(results["{'graph_type': 'ER', 'p': 0.5, 'weighted': True}"][label]))
    print(np.nanstd(results["{'graph_type': 'ER', 'p': 0.5, 'weighted': True}"][label]))
    print(np.nanmax(results["{'graph_type': 'ER', 'p': 0.5, 'weighted': True}"][label]))
    print(np.nanmin(results["{'graph_type': 'ER', 'p': 0.5, 'weighted': True}"][label]))
    print()

In [ ]:
np.argmin(results["{'graph_type': 'ER', 'p': 0.5, 'weighted': True}"]['pollner_lp_rounding'])

In [ ]:
i = 52
evaluate_model(
    meta_model=None,
    meta_model_type=None,
    base_models=[],
    instances=instances[i:i+1],
    batch_size=batch_size,
    rng=rng,
    num_realizations=1,
    baselines=['pollner_lp_rounding'],
    **baselines_kwargs
)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(ratios, bins=10)

In [ ]:
0.672 * 0.97